In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from scipy.stats.mstats import gmean

In [2]:
hvg = pd.read_csv('PBMC3k_HVG_seurat_250.csv', index_col=0)
clusters = pd.read_csv('seurat_clusters.csv', index_col=0)

In [3]:
hvg

,AAACATACAACCAC,AAACATTGAGCTAC,AAACATTGATCAGC,AAACCGTGCTTCCG,AAACCGTGTATGCG,AAACGCACTGGTAC,AAACGCTGACCAGT,AAACGCTGGTTCTT,AAACGCTGTAGCCA,AAACGCTGTTTCTG,...,TTTCAGTGTCACGA,TTTCAGTGTCTATC,TTTCAGTGTGCAGT,TTTCCAGAGGTGAG,TTTCGAACACCTGA,TTTCGAACTCTCAT,TTTCTACTGAGGCA,TTTCTACTTCCTCG,TTTGCATGAGAGGC,TTTGCATGCCTCAC
PPBP,0.000000,0.000000,0.000000,1.566387,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
LYZ,1.635873,1.962726,1.995416,4.521175,0.000000,1.726902,1.722356,1.690977,0.000000,3.339271,...,0.000000,5.462950,0.0,0.000000,0.000000,5.225879,0.000000,0.000000,0.000000,1.798457
S100A9,0.000000,0.000000,0.000000,3.838845,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,4.458743,0.0,0.000000,0.000000,5.629550,0.000000,1.937048,0.000000,1.798457
IGLL5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.378132,0.000000
GNLY,0.000000,0.000000,1.429744,0.000000,3.453545,0.000000,3.177575,0.000000,2.179642,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CLNS1A,0.000000,1.111715,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.963954,0.000000,0.0,0.000000,1.702808,1.358668,1.918229,1.937048,0.000000,0.000000
CCDC115,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FEM1B,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
IL1B,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,1.717839,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [4]:
clusters

,Cluster
AAACATACAACCAC,T
AAACATTGAGCTAC,B
AAACATTGATCAGC,T
AAACCGTGCTTCCG,CD14+ Mono
AAACCGTGTATGCG,NK
...,...
TTTCGAACTCTCAT,CD14+ Mono
TTTCTACTGAGGCA,B
TTTCTACTTCCTCG,B
TTTGCATGAGAGGC,B


In [5]:
cell_to_cluster = clusters.to_dict()['Cluster']

In [6]:
cell_types = set(list(cell_to_cluster.values()))

In [7]:
cell_type_counter = defaultdict(int)
new_cell_clusters = []
for cell_cluster in cell_to_cluster.values():
    cnt = cell_type_counter[cell_cluster]
    cell_type_counter[cell_cluster] += 1
    new_cell_clusters.append(cell_cluster + ' ' + str(cnt))

In [8]:
cell_to_cluster_new = dict(zip(cell_to_cluster.keys(), new_cell_clusters))

In [9]:
new_columns = [cell_to_cluster_new[col] for col in hvg.columns]

In [10]:
hvg.columns = new_columns

In [11]:
with open('PBMC3k_HVG_250_KNN_k20.txt', 'r') as f:
    edgelist = f.readlines()
edgelist = [e.rstrip() for e in edgelist]
edgelist = [(int(e.split()[0]), int(e.split()[1])) for e in edgelist]

In [12]:
edgelist_with_names = []
for e in edgelist:
    n1 = hvg.iloc[:, e[0]].name
    n2 = hvg.iloc[:, e[1]].name
    edgelist_with_names.append((n1, n2))

In [13]:
platelet_edges = []
for en in edgelist_with_names:
    n1 = en[0]
    n2 = en[1]
    
    if 'Platelet' in n1 or 'Platelet' in n2:
        platelet_edges.append(en)

In [14]:
len(platelet_edges)

217

In [15]:
sorted_plt = set([tuple(sorted(i)) for i in platelet_edges])

In [17]:
len(edgelist_with_names)

46422

In [18]:
sorted_edgelist = set([tuple(sorted(i)) for i in edgelist_with_names])

In [19]:
all_names = [en[0] for en in edgelist_with_names]
all_names = all_names + [en[1] for en in edgelist_with_names]
all_names = sorted(list(set(all_names)))

In [20]:
first_nodes = np.array([en[0] for en in edgelist_with_names])
second_nodes = np.array([en[1] for en in edgelist_with_names])

In [21]:
edgelist_with_names[:10]

[('T 0', 'T 16'),
 ('T 0', 'T 145'),
 ('T 0', 'T 170'),
 ('T 0', 'T 219'),
 ('T 0', 'T 248'),
 ('T 0', 'T 254'),
 ('T 0', 'T 256'),
 ('T 0', 'T 298'),
 ('T 0', 'T 340'),
 ('T 0', 'CD8 T 83')]

In [22]:
cell_to_connections = {}
for cell in all_names:
    idxs_0 = np.where(first_nodes == cell)
    connections_0 = second_nodes[idxs_0]
    
    idxs_1 = np.where(second_nodes == cell)
    connections_1 = first_nodes[idxs_1]
    
    connections = np.unique(np.concatenate((connections_0, connections_1)))
    cell_to_connections[cell] = list(np.setdiff1d(connections, np.array([cell])))

In [26]:
cell_to_connections['B 0']

['B 126',
 'B 143',
 'B 179',
 'B 200',
 'B 233',
 'B 280',
 'B 302',
 'B 311',
 'B 312',
 'B 32',
 'B 321',
 'B 33',
 'B 331',
 'B 338',
 'B 38',
 'B 60',
 'B 62',
 'B 67',
 'B 69',
 'B 74',
 'B 98']

In [27]:
count_cell_types_per_cell = {}

for cell, connections in cell_to_connections.items():
    connections_only_type = [' '.join(c.split()[:-1]) for c in connections]
    count_cell_types_per_cell[cell] = Counter(connections_only_type)

In [30]:
count_cell_types_per_cell['B 0']

Counter({'B': 21})

In [31]:
frac_cell_types_per_cell = {}

for cell, connectivities_count in count_cell_types_per_cell.items():
    total = sum(dict(connectivities_count).values())
    frac_dict = {}
    for conn_cell_type, conn_cell_count in dict(connectivities_count).items():
        frac_dict[conn_cell_type] = conn_cell_count / total
    frac_cell_types_per_cell[cell] = frac_dict

In [33]:
cell_gmeans = {}
for cell, frac in frac_cell_types_per_cell.items():
    cell_gmeans[cell] = gmean(list(dict(frac).values()))

In [35]:
frac_cell_types_per_cell['T 238']

{'B': 0.0019455252918287938,
 'CD8 T': 0.017509727626459144,
 'NK': 0.0019455252918287938,
 'T': 0.9785992217898832}

In [36]:
sorted_gmeans = {k: v for k, v in sorted(cell_gmeans.items(), key=lambda item: item[1])}

In [38]:
filtered = {}
for cell, cell_frac in frac_cell_types_per_cell.items():
    if np.all(np.array(list(dict(cell_frac).values())) < 0.7):
        filtered[cell] = count_cell_types_per_cell[cell]

In [40]:
filtered

{'CD14+ Mono 138': Counter({'CD14+ Mono': 15, 'FCGR3A+ Mono': 8}),
 'CD14+ Mono 155': Counter({'CD14+ Mono': 28, 'FCGR3A+ Mono': 15}),
 'CD14+ Mono 162': Counter({'CD14+ Mono': 14, 'FCGR3A+ Mono': 8}),
 'CD14+ Mono 190': Counter({'CD14+ Mono': 16, 'FCGR3A+ Mono': 15}),
 'CD14+ Mono 200': Counter({'CD14+ Mono': 15, 'FCGR3A+ Mono': 11}),
 'CD14+ Mono 220': Counter({'CD14+ Mono': 42,
          'FCGR3A+ Mono': 25,
          'Platelet': 1}),
 'CD14+ Mono 29': Counter({'CD14+ Mono': 15, 'DC': 25, 'FCGR3A+ Mono': 1}),
 'CD14+ Mono 299': Counter({'CD14+ Mono': 11, 'FCGR3A+ Mono': 9}),
 'CD14+ Mono 301': Counter({'CD14+ Mono': 12, 'DC': 7, 'FCGR3A+ Mono': 1}),
 'CD14+ Mono 387': Counter({'CD14+ Mono': 34, 'FCGR3A+ Mono': 22}),
 'CD14+ Mono 401': Counter({'CD14+ Mono': 22, 'DC': 8, 'FCGR3A+ Mono': 5}),
 'CD14+ Mono 420': Counter({'CD14+ Mono': 36, 'DC': 1, 'FCGR3A+ Mono': 15}),
 'CD14+ Mono 423': Counter({'CD14+ Mono': 14, 'DC': 1, 'FCGR3A+ Mono': 6}),
 'CD14+ Mono 436': Counter({'CD14+ Mono': 1

In [40]:
pd.DataFrame.from_dict(filtered, orient='index').to_csv('cell_knn_neighbours_counts_highest.csv')

In [ ]:
pd.DataFrame.from_dict(selected_dict, orient='index').to_csv('cell_knn_neighbours_fractions_highest.csv')